In [ ]:
%matplotlib inline

import os
import sys
from itertools import product
import numpy as np
np.set_printoptions(precision=4,suppress=True, linewidth=140)
import scipy

import matplotlib.pyplot as plt

from h5 import HDFArchive
from triqs.gf import *
from triqs.gf.descriptors import Fourier, Omega
from triqs.plot.mpl_interface import plt,oplot
from triqs.lattice.utils import TB_from_wannier90, k_space_path

import solid_dmft.postprocessing.plot_correlated_bands as pcb
import solid_dmft.postprocessing.maxent_sigma as sigma_maxent

In [ ]:
fermi = 12.7367
# w90 information
w90_dict = {'w90_path': './data/mlwf/',
            'w90_seed': 'lco',
            'n_orb': 1,
            'mu_tb': fermi}

# plotting options
plot_dict = {'colorscheme_bands': 'Greys',
             'colorscheme_alatt': 'Spectral_r',
             'colorscheme_kslice': 'Spectral_r'}

tb_dict = {'bands_path': [('G', 'S\''), ('S\'', 'X'), ('X', 'G'), ('G', 'Z')], 'n_k': 50,
            'G': [ 0.0,  0.0,  0.0], 'S\'': [0.25, -0.25, 0.25], 'X': [0.0,  0.0,  0.5 ], 'Z': [0.25,  0.25, -0.25] }

# Comparison to Hubbard-I solution

One feature of `solid_dmft` is the ability to seamlessly switch between different DMFT impurity solver. In this notebook we will compare the cthyb results with results of the [HubbardI solver](https://triqs.github.io/hubbardI). The `hubbardI` solver is much lighter than the cthyb solver, but numerically not exact. It neglects the coupling to the bath to calculate the self-energy, and solves the local Hamiltonian via exact diagonalization.

To use the solver we prepared a second input file called `dmft_config_hubbardI.ini`. Take a look at the file: 

In [ ]:
!cat dmft_config_hubbardI.ini

The only necessary change is to switch the `solver_type` flag to `hubbardI`, and remove some unecessary flags in the `[solver]` section that are specific to Quantum Monte Carlo solvers. Run `solid_dmft` with this input file by passing the name of the config file as parameter: 

`>>> mpirun -n 4 solid_dmft dmft_config_hubbardI.ini`

After the solver is finished (should be done in a minute or so) we load the results:

In [ ]:
with HDFArchive('b10-U3.6-hubbardI/lco_wannier.h5','r') as h5:
    print(h5['DMFT_results']['last_iter'])
    # the next two lines load all directly measured observables and the convergence metrics per iteration
    obs = h5['DMFT_results/observables']
    conv_obs = h5['DMFT_results/convergence_obs']
    # The next lines load the last iteration impurity Green's function and self-energies
    Sigma_real_freq_hI = h5['DMFT_results/last_iter/Sigma_Refreq_0']
    G_atomic_w = h5['DMFT_results/last_iter/Gimp_Refreq_0']
    G_imp_iw = h5['DMFT_results/last_iter/Gimp_freq_0']
    G_imp_tau = h5['DMFT_results/last_iter/Gimp_time_0']
    # last, the chemical potential and double counting potential
    mu_hI = h5['DMFT_results']['last_iter']['chemical_potential_post']
    dc_hI = h5['DMFT_results']['last_iter']['DC_pot']

We can now directly load the real frequency self-energy as `Sigma_real_freq_hI`, since the ED solvers can calculate the impurity Green's function directly on real frequency axis via the Lehman spectral representation, and solve the Dyson equation on the real axis. 

<i class="fa fa-gear fa-2x" style="color: #186391"></i>&nbsp; compare the convergence with results from cthyb using the code fragments of the first notebook

<i class="fa fa-gear fa-2x" style="color: #186391"></i>&nbsp; compare the self-energy with the analytically continued self-energies for the metallic and the insulating cthyb results. Compare the shapes.

In [ ]:
with HDFArchive('b10-U3.6/lco_wannier.h5','r') as h5:
    Sigma_real_freq_cthyb = h5['DMFT_results/last_iter']['Sigma_maxent_0']
    mu = h5['DMFT_results']['last_iter']['chemical_potential_post']
    dc = h5['DMFT_results']['last_iter']['DC_pot']
    
# replace XXX with your U value
with HDFArchive('b10-XXXX/lco_wannier.h5','r') as h5:
    Sigma_real_freq_cthyb_ins = h5['DMFT_results/last_iter']['Sigma_maxent_0']

In [ ]:
fig, ax = plt.subplots(1,2, sharex=True, dpi=150, figsize=(10,4))

# plot real part of the self-energy and subtract constant offset at 0.0 (Hartree shift)
# Sigma_real_freq['up_0'](0.0) evaluates the self-energy at w=0.0
ax[0].oplot((Sigma_real_freq_cthyb['up_0'] - Sigma_real_freq_cthyb['up_0'](0.0)[0,0]).real, '-', c='C0', label='cthyb met U=3.6')

# plot imaginary part of the self-energy
ax[1].oplot(-1*Sigma_real_freq_cthyb['up_0'].imag, '-', c='C0', label='cthyb met  U=3.6')

ax[0].set_ylabel(r"Re $\Sigma (\omega)$ (eV)")
ax[1].set_ylabel(r"-Im $\Sigma (\omega)$ (eV)")
ax[0].set_ylim(-10,10)
ax[1].set_ylim(0,10)
ax[-1].set_xlim(-2.5,2.5)
plt.show()

Now we can plot and compare the k-dependent spectral function:

In [ ]:
try:
    dc_hI[0]['down_0'] = dc_hI[0].pop('down')
    dc_hI[0]['up_0'] = dc_hI[0].pop('up')
except: 
    pass

In [ ]:
# we have to specify the mesh of the spectral function and pass a new keywargs object sigma_dict
w_mesh = {'window': [-4.0, 4.0], 'n_w': int(501)}
sigma_dict = {'dmft_path': 'b10-U3.6-hubbardI/lco_wannier.h5', 
              'it': 'last_iter',
              'spin': 'up_0', 
              'block' : 0 ,
              'w_mesh': w_mesh, 
              'dc': dc_hI, 
              'mu_dmft': mu_hI}

tb_bands, alatt_k_w_path, freq_dict = pcb.get_dmft_bands(with_sigma=Sigma_real_freq_hI, eta=0.1,
                                                         **w90_dict, **tb_dict, **sigma_dict)

fig, ax = plt.subplots(1,dpi=150,figsize=(7,4))
pcb.plot_bands(fig, ax, alatt_k_w_path, tb_bands, freq_dict, n_orb=w90_dict['n_orb'], 
               tb=True, alatt=True, **plot_dict)

<i class="fa fa-gear fa-2x" style="color: #186391"></i>&nbsp; calculate / plot once again the k-integrated spectral function for hubbardI and compare with your previous cthyb result. Plot also the atomic Green's function of the hubbardI solver `G_atomic_w`, which is already on the real frequency axis. Try to answer the following questions:
1) Why do the spectra of the impurity Green's function and the k-integrated lattice Green's function differ?
2) Which features of the spectral function are captured correctly for the same U value?
3) For which kind of systems does the hubbardI approximation deliver reasonable results?

Tip: When summing the lattice Green's functions for the HubbardI self-energy you have to introduce an extra broadening. Add something like `0.4j`.

In [ ]:
fig, ax = plt.subplots(1,dpi=150,figsize=(6,3))

ax.oplot(-1/np.pi*G_loc_w_cthyb.imag, label=r'A($\omega$) cthyb')

ax.oplot(-1/np.pi*G_atomic_w['up_0'].imag, label=r'A($\omega$) atomic')

ax.oplot(-1/np.pi*G_loc_w.imag, label=r'A($\omega$) loc')

ax.set_ylabel(r'A($\omega$)')
ax.set_xlabel(r'$\omega$')
ax.set_xlim(-5,5)
ax.set_ylim(0,1.1)
ax.legend()
plt.show()

## Advanced: Running a Full charge self-consistent DFT+DMFT calculation

Now that you have mastered the one-shot calculation, you can repeat the calculation with full charge self-consistency and see how the solution changes. Instructions are described in the tutorial:
[triqs.github.io/solid_dmft/tutorials/Ce2O3_csc_w90/tutorial](https://triqs.github.io/solid_dmft/tutorials/Ce2O3_csc_w90/tutorial.html)